# Installing Dependencies

In [1]:
pip install lyricsgenius

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from striprtf.striprtf import rtf_to_text
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil import parser
import matplotlib.pyplot as plt
import json
import lyricsgenius

/Users/davidcalalang/miniconda3/envs/genai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Reading Lyrics (.rtf) File and Converting to String

In [8]:
with open("When I Was Your Man.rtf") as file:
    content = file.read()
    text = rtf_to_text(content)
print(text)

"When I Was Your Man"

Same bed but it feels just a little bit bigger now
Our song on the radio but it don't sound the same
When our friends talk about you, all it does is just tear me down
'Cause my heart breaks a little when I hear your name

It all just sounds like oooooh…
Mmm, too young, too dumb to realize
That I should've bought you flowers
And held your hand
Should've gave you all my hours
When I had the chance
Take you to every party
'Cause all you wanted to do was dance
Now my baby's dancing
But she's dancing with another man

My pride, my ego, my needs, and my selfish ways
Caused a good strong woman like you to walk out my life
Now I'll never, never get to clean up the mess I made, ohh…
And it haunts me every time I close my eyes

It all just sounds like oooooh…
Mmm, too young, too dumb to realize
That I should've bought you flowers
And held your hand
Should've gave you all my hours
When I had the chance
Take you to every party
'Cause all you wanted to do was dance
Now my bab

# Using Genius API to Import Lyrics (main resource: https://github.com/johnwmillr/LyricsGenius#usage)

In [7]:
import os
import getpass

In [8]:
os.environ["GENIUS_API_KEY"] = getpass.getpass()
genius = lyricsgenius.Genius(os.environ.get("GENIUS_API_KEY"))

In [9]:
album = genius.search_album("Unorthodox Jukebox", "Bruno Mars")
album.save_lyrics()
# a .json file is saved in the directory containing information for the whole album

Searching for "Unorthodox Jukebox" by Bruno Mars...
Skipping file save.



In [3]:
with open("Lyrics_UnorthodoxJukebox.json") as f:
    lyrics = json.load(f)

In [77]:
# lyrics now stores the .json file as a python string, contains a lot of different fields, key ones for our purposes
# are 'tracks', and within 'tracks', 'song'
print(lyrics['tracks'][3]['number'])

4


In [67]:
# To store our lyrics, we'll be using python dictionaries with two keys each to store the song title and their lyrics
# We use string splicing to deal with ignoring some extra meta-info included within the 'lyrics' field

song_lyrics_dict = {"song": [], "lyrics": []}

for track in lyrics['tracks']:
    new_lyrics = track['song']['lyrics']
    start_index = new_lyrics.find('[Verse 1]')
    new_lyrics = new_lyrics[start_index:]
    
    end_index = new_lyrics.find('Embed')
    new_lyrics = new_lyrics[:(end_index-2)]
    
    song_lyrics_dict["song"].append(track['song']['title_with_featured'])
    song_lyrics_dict["lyrics"].append(new_lyrics)

In [73]:
# Example of the info stored within our python dictionary
print(song_lyrics_dict["song"][4])
print("-------")
print(song_lyrics_dict["lyrics"][4][:50])

Moonshine
-------
[Verse 1]
Hello
You know, you look even better tha


# Sentiment Analyzer Model